In [ ]:
import os
import pandas as pd
import sys
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

from utils_plotting import create_dirs
from data_extraction import load_data_to_dataframe

In [7]:
# ------ Settings ------
# => These variables should be set to the desired settings!

# The files in the directory should have names of the form:
# "{dataset}-factor-{downsample factor}-{any remaining information}"
directory = ''

# The algorithms to consider, needs to match Excel header
algorithms = [
    'Change Vector Analysis',
    'Iterative Reweighted MAD'
]

# ---- The metrics to consider ----

# Given in the Excel file
metrics_precalc = []

# Need to be calculated here -> assumes that the Excel file contains TP, FP, TN and FN
metrics_calc = [
    'OA',
    'Recall',
    'Precision',
    'FMeasure',
    'Kappa'
]

In [3]:
# Load the data
df = load_data_to_dataframe(directory, metrics_calc)

# Create the directories
metrics = metrics_calc + metrics_precalc
dir_dict = create_dirs(directory, metrics)

## Heatmap

Reference: https://medium.com/@szabo.bibor/how-to-create-a-seaborn-correlation-heatmap-in-python-834c0686b88e

In [4]:
def createHeatmap(data, attributes, title, directory, subfolder, figname):
    data = data[attributes]
    corr = data.corr()
    mask = np.triu(np.ones_like(corr)) 

    plt.figure(figsize=(len(attributes) + 1, len(attributes)))
    hm = sns.heatmap(corr, vmin = -1, vmax = 1, center = 0, annot=True, fmt='.1f', 
                     square=True, linewidths=.5, cmap = 'coolwarm', mask=mask)
    hm.set_title(title, fontdict={'fontsize':12}, pad=12)
    
    folder = os.path.join(directory, subfolder)
    if not os.path.exists(folder):
        os.makedirs(folder)

    plt.savefig(os.path.join(folder, figname + '.png'))

In [ ]:
attrs = ['refWidth', 'refHeight', 'scaledRes', 'OA', 'Precision', 'Recall']

createHeatmap(
    df.loc[(df['name'] == 'Change Vector Analysis') & (df['threAlg'] == 'K-Means')],
    attrs,
    'CVA K-Means all',
    dir_dict['heatmap_dir'],
    'Change Vector Analysis',
    'cva-kmeans-all'
)

createHeatmap(
    df.loc[(df['name'] == 'Iterative Reweighted MAD') & (df['threAlg'] == 'K-Means')],
    attrs,
    'IRMAD K-Means all',
    dir_dict['heatmap_dir'],
    'Iterative Reweighted MAD',
    'irmad-kmeans-all'
)

createHeatmap(
    df.loc[(df['name'] == 'Change Vector Analysis') & (df['threAlg'] == 'K-Means') & 
           (df['dataset'] == 'LEVIRCDDataset')],
    attrs,
    'CVA K-Means LEVIR',
    dir_dict['heatmap_dir'],
    'Change Vector Analysis',
    'cva-kmeans-levir'
)

createHeatmap(
    df.loc[(df['name'] == 'Iterative Reweighted MAD') & (df['threAlg'] == 'K-Means') & 
           (df['dataset'] == 'LEVIRCDDataset')],
    attrs,
    'IRMAD K-Means LEVIR',
    dir_dict['heatmap_dir'],
    'Iterative Reweighted MAD',
    'irmad-kmeans-levir'
)

createHeatmap(
    df.loc[(df['name'] == 'Change Vector Analysis') & (df['threAlg'] == 'K-Means') & 
           (df['dataset'] == 'OSCDDataset')],
    attrs,
    'CVA K-Means OSCD',
    dir_dict['heatmap_dir'],
    'Change Vector Analysis',
    'cva-kmeans-oscd'
)

createHeatmap(
    df.loc[(df['name'] == 'Iterative Reweighted MAD') & (df['threAlg'] == 'K-Means') & 
           (df['dataset'] == 'OSCDDataset')],
    attrs,
    'IRMAD K-Means OSCD',
    dir_dict['heatmap_dir'],
    'Iterative Reweighted MAD',
    'irmad-kmeans-oscd'
)

In [ ]:
attrs = ['refWidth', 'refHeight', 'scaledRes', 'OA', 'Precision', 'Recall']

levir_imgs = ['test_89', 'test_77', 'test_38', 'test_39', 'test_59']
oscd_imgs = ['milano-cm', 'valencia-cm', 'dubai-cm']

imgs = levir_imgs + oscd_imgs
algs = {
    'Change Vector Analysis': 'CVA',
    'Iterative Reweighted MAD': 'IRMAD',
}

for img in imgs:
    for alg, short_alg in algs.items():
        createHeatmap(
            df.loc[(df['name'] == alg) & (df['threAlg'] == 'K-Means') & (df['refName'] == img)],
            attrs,
            short_alg + ' K-Means ' + img,
            dir_dict['heatmap_dir'],
            img,
            short_alg + '-kmeans-' + img
        )

## Means

In [ ]:
def plotMean(data, x_attr, y_attr, x_label, y_label):
    return